In [549]:
!git clone https://github.com/actondev/wavelet-denoiser.git

Cloning into 'wavelet-denoiser'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 124, done.
remote: Total 124 (delta 0), reused 0 (delta 0), pack-reused 124 (from 1)
Receiving objects: 100% (124/124), 408.23 KiB | 4.30 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [ ]:
!pip install -r wavelet-denoiser/requirements.txt

In [451]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Wav2Vec2ForCTC, Wav2Vec2Processor
from sklearn.metrics import accuracy_score, f1_score
import torch
from datasets import load_dataset, Audio
import numpy as np
from speechbrain.inference.separation import SepformerSeparation as separator
torch.manual_seed(42)

from typing import Dict
import re
from IPython.display import Audio as play_audio
import os
import tempfile
import subprocess
import soundfile as sf

In [2]:
llm_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(llm_name)

In [509]:
### инициализация классификатора из текста

In [3]:
llm = AutoModelForCausalLM.from_pretrained(
    llm_name, 
    torch_dtype=torch.bfloat16,
    device_map="balanced",
    max_memory={0: '10GB', 1: '10GB', 2: '10GB'}
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [33]:
prompt = \
'''
Задача: Классифицируй текстовые команды пилота/бортового ИИ в один из 9 классов на основе их содержания. В ответе не давай обоснования и названия класса, а ТОЛЬКО НОМЕР.

Примеры:

1. Запрос: "Опустись на уровень 4.2 километра"  
   Класс: 1 (Снизить высоту)

2. Запрос: "Набери 7.5 км высоты"  
   Класс: 2 (Увеличить высоту)

3. Запрос: "Тормози до 300 км/ч"  
   Класс: 3 (Снизить скорость)

4. Запрос: "Ускорение до 650 км/час"  
   Класс: 4 (Увеличить скорость)

5. Запрос: "Подготовить закрылки и шасси"  
   Класс: 5 (Запустить подготовку к посадке)

6. Запрос: "Проверить топливные системы перед стартом"  
   Класс: 6 (Начать проверку систем перед взлетом)

7. Запрос: "Полный отчет о работоспособности"  
   Класс: 7 (Проверить состояние всех систем)

8. Запрос: "Вывести данные о воздушной скорости"  
   Класс: 8 (Отобразить текущую скорость)

9. Запрос: "Активировать карту маршрута"  
   Класс: 9 (Открыть систему навигации)

---

Новый запрос для классификации: "{command_text}"  
Класс (номер):
'''

In [541]:
def combine_texts(text1: str, text2: str) -> str:
    
    messages = [
        {"role": "system", "content": "Твоя задача - быть полезным диалоговым ассистентом."},
        {"role": "user", "content": 'склей следующие 2 команды в одну, а также исправь ошибки, учти, что команда из контекста управления самолётом "{text1}", "{text2}", в ответе укажи ТОЛЬКО финальный результат'.replace('{text1}', text1).replace('{text2}', text2)}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(llm.device)
    
    generated_ids = llm.generate(
        **model_inputs,
        max_new_tokens=64,
        do_sample=False
    )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    match = re.search(r'"(.*?)"', response)
    if match:
        response = match.group(1)
    
    return response

In [545]:
def text_classify(text: str) -> str:

    messages = [
        {"role": "system", "content": "Твоя задача - быть полезным диалоговым ассистентом."},
        {"role": "user", "content": prompt.replace('{command_text}', text)}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(llm.device)
    generated_ids = llm.generate(
        **model_inputs,
        max_new_tokens=24,
        do_sample=False
    )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [35]:
text_classify('Увеличить скорость до 960 км/ч')

'4'

In [510]:
### инициализация модели speech to text

In [28]:
speech_to_text_model_name = "NLPVladimir/wav2vec2-large-xls-r-300m-ru"  

processor = Wav2Vec2Processor.from_pretrained(
    speech_to_text_model_name,
    device='cuda:0'
)

speech_to_text_model = Wav2Vec2ForCTC.from_pretrained(
    speech_to_text_model_name,
    device_map='cuda:0',
)

In [29]:
def get_text_from_raw_speech(audio: Dict) -> str:
    input_values = processor(
        audio['array'], 
        sampling_rate=audio['sampling_rate'], 
        return_tensors="pt"
    ).input_values.to(speech_to_text_model.device)
    
    with torch.no_grad():
        logits = speech_to_text_model(input_values).logits
    
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription

In [9]:
dataset = load_dataset("NLPVladimir/aviation-command-dataset-ru")

In [298]:
audio_example = dataset['train'][4]['audio']

In [299]:
play_audio(audio_example['array'], rate=audio_example['sampling_rate'])

In [300]:
get_text_from_raw_speech(audio_example)

'увеличить весотой полета пятитеси восьми тысячи метры паджи'

In [511]:
### классификация без denoising модели

In [36]:
def classify_command_from_speech_raw(audio: Dict):
    
    command_text = get_text_from_raw_speech(audio)
    predicted_class = text_classify(command_text)

    return predicted_class

In [37]:
classify_command_from_speech_raw(audio_example)

'6'

In [48]:
def add_raw_answer(batch):
    batch["raw_answer"] = classify_command_from_speech_raw(batch['audio'])
    return batch

In [ ]:
dataset['train'] = dataset['train'].map(add_raw_answer)

Map:   0%|          | 0/1182 [00:00<?, ? examples/s]

In [69]:
def parse_answer(batch):
    batch["raw_answer_num"] = re.search(r'\d+', batch["raw_answer"])[0]
    return batch

def parse_label(batch):
    batch["label_num"] = re.search(r'\d+', str(batch["label"] + 1))[0]
    return batch

dataset['train'] = dataset['train'].map(parse_answer)
dataset['train'] = dataset['train'].map(parse_label)

Map:   0%|          | 0/1182 [00:00<?, ? examples/s]

Map:   0%|          | 0/1182 [00:00<?, ? examples/s]

In [ ]:
### Метрики без denoising модели

In [76]:
true_labels = dataset['train']['label_num']
predictions = dataset['train']['raw_answer_num']

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.2f}")

weighted_f1 = f1_score(true_labels, predictions, average='weighted')
print(f"Weighted F1 Score: {weighted_f1:.2f}")

Accuracy: 0.56
Weighted F1 Score: 0.59


In [512]:
### denoising модель, использующая FFT преобразования

In [479]:
denoising_model = separator.from_hparams(source="speechbrain/rescuespeech_sepformer", run_opts={"device":"cuda"})

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/rescuespeech_sepformer' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/rescuespeech_sepformer' if not cached
INFO:speechbrain.utils.fetching:Fetch encoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/rescuespeech_sepformer' if not cached
INFO:speechbrain.utils.fetching:Fetch masknet.ckpt: Fetching from HuggingFace Hub 'speechbrain/rescuespeech_sepformer' if not cached
INFO:speechbrain.utils.fetching:Fetch decoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/rescuespeech_sepformer' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: encoder, masknet, decoder


In [553]:
def process_audio_fft(example):
    with tempfile.TemporaryDirectory() as tmpdir:
        input_path = os.path.join(tmpdir, "tmp_input.wav")

        sf.write(input_path, example["array"], example["sampling_rate"])
        est_sources = denoising_model.separate_file(path=input_path)
        output_array = est_sources[:, :, 0].detach()[0].cpu().numpy()
        
        return {"array": output_array, "sampling_rate": example["sampling_rate"]}

In [554]:
def process_audio_wavelet(example):
    with tempfile.TemporaryDirectory() as tmpdir:
        
        input_path = os.path.join(tmpdir, "tmp_input.wav")
        output_path = os.path.join(tmpdir, "tmp_output.wav")
        
        sf.write(input_path, example["array"], example["sampling_rate"])
        
        try:
            subprocess.run(
                ["python3", "wavelet-denoiser/src/denoiser-argument.py", "-i", input_path, "-o", output_path],
                check=True,
                capture_output=True
            )
            
            output_array, sr = sf.read(output_path)
            
            if sr != example["sampling_rate"]:
                raise ValueError("Sampling rate mismatch")
            
            return {"array": output_array, "sampling_rate": sr}
            
        except Exception as e:
            return example

In [555]:
filtered_audio_fft = process_audio_fft(dataset['train'][2]['audio'])

In [557]:
filtered_audio_wavelet = process_audio_wavelet(dataset['train'][2]['audio'])

In [491]:
play_audio(dataset['train'][2]['audio']['array'], rate=16_000)

In [492]:
play_audio(filtered_audio_fft['array'], rate=16_000)

In [560]:
play_audio(filtered_audio_wavelet['array'], rate=16_000)

In [513]:
### Классификация с denoising моделями

In [584]:
def classify_command_from_speech_denoise_fft(audio: Dict):
    
    command_text_denoised = get_text_from_raw_speech(process_audio_fft(audio))
    command_text_raw = get_text_from_raw_speech(audio)
    combined_text = combine_texts(command_text_raw, command_text_denoised)
    predicted_class = text_classify(combined_text)

    return predicted_class


def add_denoise_answer_fft(batch):
    batch["denoise_answer"] = classify_command_from_speech_denoise_fft(batch['audio'])
    return batch


def classify_command_from_speech_denoise_wavelet(audio: Dict):
    
    command_text_denoised = get_text_from_raw_speech(process_audio_wavelet(audio))
    command_text_raw = get_text_from_raw_speech(audio)
    combined_text = combine_texts(command_text_raw, command_text_denoised)
    predicted_class = text_classify(combined_text)

    return predicted_class
    

def add_denoise_answer_wavelet(batch):
    batch["denoise_answer"] = classify_command_from_speech_denoise_wavelet(batch['audio'])
    return batch


def parse_denoise_answer(batch):
    batch["denoise_answer_num"] = re.search(r'\d+', batch["denoise_answer"])[0]
    return batch

In [585]:
classify_command_from_speech_denoise_fft(dataset['train'][2]['audio'])

'9'

In [586]:
classify_command_from_speech_denoise_wavelet(dataset['train'][2]['audio'])

/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `70` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


'7'

In [ ]:
dataset['train'] = dataset['train'].map(add_denoise_answer_fft)

Map:   0%|          | 0/1182 [00:00<?, ? examples/s]

In [ ]:
dataset['train'] = dataset['train'].map(parse_denoise_answer)

In [ ]:
### Метрики с denoising fft моделью

In [590]:
true_labels = dataset['train']['label_num']
predictions = dataset['train']['denoise_answer_num']

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy with fft denoising: {accuracy:.2f}")

weighted_f1 = f1_score(true_labels, predictions, average='weighted')
print(f"Weighted F1 Score with fft denoising: {weighted_f1:.2f}")

Accuracy with fft denoising: 0.61
Weighted F1 Score with fft denoising: 0.63


In [ ]:
dataset['train'] = dataset['train'].map(add_denoise_answer_wavelet)

Map:   0%|          | 0/1182 [00:00<?, ? examples/s]

/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `70` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/home/vshilonosov/miniconda3/lib/python3.12/sit

In [568]:
dataset['train'] = dataset['train'].map(parse_denoise_answer)

Map:   0%|          | 0/1182 [00:00<?, ? examples/s]

In [569]:
### Метрики с Wavelet denoising

In [596]:
true_labels = dataset['train']['label_num']
predictions = dataset['train']['denoise_answer_num']

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy with wavelet denoising: {accuracy:.2f}")

weighted_f1 = f1_score(true_labels, predictions, average='weighted')
print(f"Weighted F1 Score with wavelet denoising: {weighted_f1:.2f}")

Accuracy with wavelet denoising: 0.57
Weighted F1 Score with wavelet denoising: 0.59
